# rrFpy_pandas (Read ROOT File using python in pandas manner) package 

(c) Alexey Luchinsky

This package could be useful for testing newly created EvtGen models. Supplied with EvtGen program **simpleEvtGenRO.exe** is nice with producing some output files, but the format of these files are rather complicated, a special program should be created to read it and to extract the histograms from it

The supplied package makes this process very clean, easy, and fun.

To use it you should only import one package

In [ ]:
from  rrfpy.rrfpy_pandas import *

For further demonstration of it's functionality we should also import some other standard python packages:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as ps
import plotly.graph_objects as go
from IPython.display import display, Latex
import plotly.offline as pyo
pyo.init_notebook_mode()
import sys

Let us start


## $\tau$ Decays

We start with considering very simple decay: $\tau^- \to e^- \bar{\nu}_e \nu_\tau$

The file we are analyzing was created in the **build/** directory using the command

    ./simpleEvtGenRO.exe tau- ../src/tau_enu.dec  100000
    cp evtOutput.root evtOutput_tau.root
    
Let is download it first

In [ ]:
rrF = rrFpy_pandas("../c++/build/evtOutput_tau.root")

We can see that 100000 events were loaded

In [ ]:
rrF.size()

and all these events correspond to shown above $\tau$-lepton decay

In [ ]:
rrF["reac"].value_counts()

The decay tree of the specific event can be easily viewed:

In [ ]:
rrF.PDT()

This method can be configured in several ways, you are welcome to experiment with it

The distributions over various kinematical vars can be extracted using **["var"]** notation. Here is the energy of the first produced particle (i.e. electron), for example

In [ ]:
distE = rrF["E_1"]
distE

The result is usual **pandas.Series** object, so you can do whatever you want with is.

Calculate the mean:

In [ ]:
distE.mean()

or even plot the histogram

In [ ]:
distE.hist(bins=50, density = True)
# analytical results
mtau = rrF["m_0"].mean()
e = np.linspace(0, mtau/2, 100)
plt.plot(e, 16/mtau**3*(3-4*e/mtau)*e**2, 'r', linewidth = 5);
# final touches
plt.xlabel(r"$E_e,\,\mathrm{GeV}$")
plt.ylabel(r"$d\mathrm{Br}/dE_e,\,\mathrm{GeV}^{-1}$")
plt.title(rrF["reac"].iloc[0])
plt.show()

Note that the result perfectly agrees with the textbook variant
$$\frac{d\mathrm{Br}}{dE}= \frac{16}{m_\tau^3}\left[3-\frac{4E}{m_\tau}\right]E^2,$$
that is shown with red line in the above plot

All other distributions can also be easily constricted. Here, for example, are distributions over different square masses

In [ ]:
rrF[["m2_12", "m2_13", "m2_23"]].plot.hist(bins=50, histtype = 'step', density = True)
plt.xlabel(r"$m^2,\,\mathrm{GeV^2}$")
plt.ylabel(r"$d\mathrm{Br}/dm^2,\,\mathrm{GeV}^{-2}$")
plt.title(rrF["reac"].iloc[0])
plt.show()
plt.show()

Note that several distributions can be extracted in one command

As expected, the relation
$$s+t+u = \sum_i m_i^2$$
does hold for all events

In [ ]:
np.sqrt(rrF["m2_12"]+rrF["m2_13"] + rrF["m2_23"])

Any cuts can easily be imposed to the data set

In [ ]:
rrFcut = rrF.cut("E_1>0.4")
print("%d events survived after the cut (%d %%)" % (rrFcut.size(), 100*rrFcut.size()/rrF.size()) )
rrFcut[["m2_12", "m2_13", "m2_23"]].plot.hist(bins=50, alpha = 0.5, density = True)
plt.show()

## $B_c$ Decays

Let us try the package on more serious example: $B_c^+$ meson decays.

The file we will be using here is created in **../build/** directory by the commands

    ./simpleEvtGenRO.exe B_c+ ../src/Bc.dec  100000
    cp evtOutput.root evtOutput_Bc.root
    
This can take some time and several **probmax** errors will be printed, it is normal

As you can see from the decay file

    noPhotos
    Decay B_c+
    1. J/psi pi+ pi+ pi- BC_VHAD 1;
    1. J/psi pi+ pi+ pi- pi- pi+ BC_VHAD 1;
    1. J/psi K+ K- pi+ BC_VHAD 1;
    1. J/psi K+ pi+ pi- BC_VHAD 1;
    Enddecay
    End
    
the whole bunch of reactions is stored in the resulting ROOT

Let us first load it

In [ ]:
rrF = rrFpy_pandas()
rrF.load_ROOT("../c++/build/evtOutput_Bc.root")

Here is the list of all reactions (with number of decays for each of them)

In [ ]:
reacs = rrF["reac"].value_counts()
print(reacs)

This data can be represented in nice graphical way

In [ ]:
reacs.plot.barh()
plt.xlabel("# of records")
plt.ylabel("reaction")
plt.show()


First we will discuss the $B_c^+ \to J/\psi \pi^+ \pi^+ \pi^-$ decay. To select the corresponding events we can use the **cut()** method (note that PDG codes for $\pi^+$ and $\pi^-$ particles are 211 and -211 respectively)

In [ ]:
rrf_3pi = rrF.cut(["ntr=5", "id_2=211","id_3=211","id_4=-211"])
rrf_3pi["reac"].value_counts()

As you can see, only the reaction we are interested in is left.

The same result can be obtained with **filter** method, that accepts a boolean list as an argument (see the similar mask functionality of the **pandas.DataFrame** objects)

In [ ]:
rrf_3pi  = rrF.filter( (rrF["ntr"]==5) & (rrF["name_2"] == "pi+") & (rrF["name_3"]=="pi+") & (rrF["name_4"]=="pi-") )
rrf_3pi["reac"].value_counts()

Sometimes the last notation is not very convenient, but it is much more powerful then the **cut()** method

In any case we can extract from the resulting object the distribution over any kinematic variable using [] notation

In [ ]:
rrf_3pi["m_23"]

The result is **pandas.Series** object, so you can do whatever you want with it.

Calculate the mean, for example

In [ ]:
rrf_3pi["m_23"].mean()

Or plot it

In [ ]:
rrf_3pi["m2_23"].hist(bins=30)
plt.xlabel(r"$m_{23}$")
plt.show()

You can use the whole python functionality, so very nice plots can be easily created in small time

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1)
rrf_3pi = rrF.cut(["ntr=5","id_2=211"])
variables = {'m_23':r'$\pi^+\pi^+$', 'm_24':r'$\pi^+\pi^-$', 'm_34':r'$\pi^+\pi^-$'}
for var, name in variables.items():
    rrf_3pi[var].hist(label=name, alpha=0.5, bins=50)
fig.suptitle(rrf_3pi["reac"].drop_duplicates().iloc[0])
ax.axvline(x=0.77, linestyle = '--', color = 'r')
plt.legend()
plt.show()

In the above plot you can easily see that distributions over two $\pi^+\pi^-$ masses are the same (two $\pi^+$ mesons are identical) and show the peak at $m_{\pi\pi}\sim m_\rho \approx 770$ MeV (is shown with the red dashed line).

In the case of $m_{\pi^+\pi^+}$ distribution such a peak is absent.

The plot can be made more beautiful and interactive with the **pyplot** package. Now you can use your mouse cursor to see the exact position of the peak (pay attention to the hover text)

In [ ]:
rrf_3pi = rrF.cut(["ntr=5","id_2=211"])
data = rrf_3pi
variables = {'m_23':r'$\pi^+\pi^+$', 'm_24':r'$\pi^+\pi^-$', 'm_34':r'$\pi^+\pi^-$'}
fig = go.Figure()
for var, name in variables.items():
    fig.add_trace(go.Histogram(x=data[var], name = name, histnorm='probability density'))
fig.update_layout(barmode='overlay',
                  xaxis_title=r"$m_{\pi\pi}, \mathrm{GeV}$",
                yaxis_title="$d\mathrm{Br}/dm_{\pi\pi}, \mathrm{GeV}^{-1}$",
                 title = data["reac"].drop_duplicates().iloc[0])
fig.update_traces(opacity=0.5)
fig.show()

With minimal modifications we draw the similar distributions for other reactions

In [ ]:
rrf_Kpipi = rrF.cut(["ntr=5","id_2=321","id_3=211"])
variables = {'m_23':r'$K^+\pi^+$', 'm_24':r'$K^+\pi^-$', 'm_34':r'$\pi^+\pi^-$'}
fig = go.Figure()
for var, name in variables.items():
    fig.add_trace(go.Histogram(x=rrf_Kpipi[var], name = name, histnorm='probability density'))
fig.update_layout(barmode='overlay',
                  xaxis_title=r"$m_{\pi\pi}, \mathrm{GeV}$",
                yaxis_title="$d\mathrm{Br}/dm_{\pi\pi}, \mathrm{GeV}^{-1}$",
                 title = rrf_Kpipi["reac"].drop_duplicates().iloc[0])
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
rrf_KKpi = rrF.cut(["ntr=5","id_2=321","id_3=-321"])
variables = {'m_23':r'$K^+K^-$', 'm_24':r'$K^+\pi^+$', 'm_34':r'$K^-\pi^+$'}
fig = go.Figure()
for var, name in variables.items():
    fig.add_trace(go.Histogram(x=rrf_KKpi[var], name = name, histnorm='probability density'))
fig.update_layout(barmode='overlay',
                  xaxis_title=r"$m_{\pi\pi}, \mathrm{GeV}$",
                yaxis_title="$d\mathrm{Br}/dm_{\pi\pi}, \mathrm{GeV}^{-1}$",
                 title = rrf_KKpi["reac"].drop_duplicates().iloc[0])
fig.update_traces(opacity=0.5)
fig.show()

## Cleaning the Decays

Sometimes the analyzed datasets are very complicated and include decay chains. In this cases it could be useful and interesting to remove all decayed particles from them and leave only the stable ones. There is a special method for it

Let us consider the decay file **c++/src/chi.dec**:

    noPhotos
    Decay chi_c2
    0.195000000 gamma   J/psi  PHSP;
    
    Enddecay
    Decay J/psi
    0.059400000 e+      e-   PHOTOS   VLL;
    0.005600000 rho0    pi0  PARTWAVE 0.0 0.0 1.0 0.0 0.0 0.0;
    
    Enddecay
    Decay rho0
    1.000    pi+ pi-   VSS;
    Enddecay

    End

As you can see, it describes $\chi_{c2}$ meson's decays via two decay chains:
* $\chi_{c2} \to \gamma J/\psi \to \gamma e^+ e^-$
* $\chi_{c2} \to \gamma J/\psi \to \gamma \rho^0 \pi^0 \to \gamma \pi^+ \pi^- \pi^0$

In both cases $J/\psi$ is decaying, in the second case $\rho^0$ meson decays also.

The corresponding ROOT file can be created in the **c++/build** directory by the commands

     ./simpleEvtGenRO.exe chi_c2 ../src/chi.dec  100000
     mv evtOutput.root evtOutput_chi2.root
     
Let us load it

In [ ]:
rrF = rrFpy_pandas("../c++/build/evtOutput_chi2.root")
print(rrF.size()," events were loaded")
rrF["reac"].value_counts()

As we have expected, there are two types of the reaction and in each case all particles (both decayed and survived) 
are stored in the history.

To see it clearly we can look at the decay trees

In [ ]:
rrF.cut("ntr=5").PDT(iEv=0)

In [ ]:
rrF.cut("ntr=7").PDT(iEv=0)

You can also look at the decay tree of the specific particle ($J/\psi$, for example):

In [ ]:
rrF.cut("ntr=7").PDT(i=2, decay_char="->")

Let us draw the distribution over the $\rho^0$ meson mass, i.e. $m_{\pi^+\pi^-}$:

In [ ]:
rrF.cut("ntr = 7")["m_56"].hist(bins=30)
plt.show()

We clearly see the peak caused by the resonance particle, but we needed to figure out that indexes of two final $\pi$-mesons in the record are 5 and 6.

We can clean the record and remove all decayed particles:

In [ ]:
rrf_cleaned = rrF.cut("ntr=7").remove_decayed()
print(rrf_cleaned["reac"].value_counts())
print("=======")
rrf_cleaned.PDT()

Now there are only 4 particles in the dataset, the stable particles. All kinematical information is saved, so we can draw the same distribution:

In [ ]:
rrf_cleaned["m_34"].hist(bins = 30)
plt.show()

Here is the mass of the $J/\psi$ meson

In [ ]:
rrf_cleaned["m_123"]

And here is the mass of the original particle

In [ ]:
rrf_cleaned["m_0123"]

Note that the size of the object has reduced significantly:

In [ ]:
print("Original: %d MB" % int(sys.getsizeof(rrF._df_pivoted)/2**20))
print("Cleaned: %d MB" % int(sys.getsizeof(rrf_cleaned._df_pivoted)/2**20))

There are several reasons for this:

* Number of stored particles is reduced
* In the cleaned dataset for each particle we store only momentum and nTrk information, no coordinates, mothers, daughters, etc